# 개요

# Prepare

# 1. 초기 세팅

## Library & data load

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [46]:
# 처리용 df와 오리지널 버전 남겨놓음
df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202110.csv",encoding='euc_kr')
org_df = pd.read_csv("/Users/luci031/Downloads/auction/data_origin/auction_202110.csv",encoding='euc_kr')

# Pre-Processing

## Basic info

In [47]:
df.head()

,품목명,단위,등급,가격,산지,친환경구분,입력일
0,[버섯]팽이버섯,5kg,특(1등),5500,경북 청도군,무농약,20210930
1,[버섯]팽이버섯,5kg,특(1등),4000,전남 무안군,무농약,20210930
2,[버섯]팽이버섯,5kg,특(1등),10000,경북 청도군,우수농산물,20210930
3,[버섯]팽이버섯,5kg,상(2등),5000,전남 나주시,우수농산물,20210930
4,[버섯]팽이버섯,5kg,특(1등),8000,경북 청도군,우수농산물,20210930


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510896 entries, 0 to 510895
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   품목명     510896 non-null  object
 1   단위      510896 non-null  object
 2   등급      510896 non-null  object
 3   가격      510896 non-null  int64 
 4   산지      509829 non-null  object
 5   친환경구분   510896 non-null  object
 6   입력일     510896 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 27.3+ MB


In [49]:
# 산지 제외하고는 결측치 없는 것으로 확인
df.isnull().sum()

품목명         0
단위          0
등급          0
가격          0
산지       1067
친환경구분       0
입력일         0
dtype: int64

- 결측치 처리

In [50]:
###
# 양이 많지 않으므로 결측치 제거
df = df.dropna(axis=0)

- 영문명으로 전환

In [51]:
df = df.rename(columns = {'품목명':'prd',
                          '단위':'scale','등급':'class',
                          '가격':'price',
                          '산지':'origin',
                          '친환경구분':'eco',
                          '입력일':'reg_date'})

## column 별 정보

### class

In [52]:
# 이름 모두 영어로 변경
# 9등과 없음 모두 9로 변경
df['class'].value_counts().index

Index(['특(1등)', '상(2등)', '보통(3등)', '9등(등외)', '없음', '4등', '5등', '6등', '7등',
       '8등'],
      dtype='object')

In [53]:
df111 = pd.DataFrame([['특(1등)', 1], ['상(2등)',2], ['보통(3등)',3], ['9등(등외)',9], ['없음',9], ['5등',5], ['4등',4], ['6등',6], ['7등',7],['8등',8]],columns=['class','new_class'])

In [54]:
df2 = df.merge(df111)
df2 = df2.drop(columns='class')

In [55]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[버섯]팽이버섯,5kg,5500,경북 청도군,무농약,20210930,1
1,[버섯]팽이버섯,5kg,4000,전남 무안군,무농약,20210930,1
2,[버섯]팽이버섯,5kg,10000,경북 청도군,우수농산물,20210930,1
3,[버섯]팽이버섯,5kg,8000,경북 청도군,우수농산물,20210930,1
4,[버섯]팽이버섯,5kg,9500,경북 청도군,우수농산물,20210930,1
...,...,...,...,...,...,...,...
509824,[호박]늙은호박,10kg,800,전남 해남군,일반,20211029,8
509825,[감자]감자(감자),20kg,28000,강원 홍천군,일반,20211030,8
509826,[감자]감자(감자),20kg,25000,강원 홍천군,일반,20211030,8
509827,[감자]감자(감자),20kg,24000,강원 홍천군,일반,20211030,8


### eco

In [56]:
# 영문명으로 전환
# 전환기 -> 유기농 재배로 전환중
df2.loc[df2["eco"]=="일반","eco"] = "normal"
df2.loc[df2["eco"]=="우수농산물","eco"] = "good_prd"
df2.loc[df2["eco"]=="무농약","eco"] = "ecofriendly"
df2.loc[df2["eco"]=="유기농","eco"] = "organic"
df2.loc[df2["eco"]=="품질인증","eco"] = "certified"
df2.loc[df2["eco"]=="전환기","eco"] = "ing"

In [57]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[버섯]팽이버섯,5kg,5500,경북 청도군,ecofriendly,20210930,1
1,[버섯]팽이버섯,5kg,4000,전남 무안군,ecofriendly,20210930,1
2,[버섯]팽이버섯,5kg,10000,경북 청도군,good_prd,20210930,1
3,[버섯]팽이버섯,5kg,8000,경북 청도군,good_prd,20210930,1
4,[버섯]팽이버섯,5kg,9500,경북 청도군,good_prd,20210930,1
...,...,...,...,...,...,...,...
509824,[호박]늙은호박,10kg,800,전남 해남군,normal,20211029,8
509825,[감자]감자(감자),20kg,28000,강원 홍천군,normal,20211030,8
509826,[감자]감자(감자),20kg,25000,강원 홍천군,normal,20211030,8
509827,[감자]감자(감자),20kg,24000,강원 홍천군,normal,20211030,8


### reg_date

In [58]:
df2['reg_date'] = df2['reg_date'].astype('str')

In [59]:
df2['reg_date'] = pd.to_datetime(df2['reg_date'].astype('str'),format='%Y%m%d')

In [60]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[버섯]팽이버섯,5kg,5500,경북 청도군,ecofriendly,2021-09-30,1
1,[버섯]팽이버섯,5kg,4000,전남 무안군,ecofriendly,2021-09-30,1
2,[버섯]팽이버섯,5kg,10000,경북 청도군,good_prd,2021-09-30,1
3,[버섯]팽이버섯,5kg,8000,경북 청도군,good_prd,2021-09-30,1
4,[버섯]팽이버섯,5kg,9500,경북 청도군,good_prd,2021-09-30,1
...,...,...,...,...,...,...,...
509824,[호박]늙은호박,10kg,800,전남 해남군,normal,2021-10-29,8
509825,[감자]감자(감자),20kg,28000,강원 홍천군,normal,2021-10-30,8
509826,[감자]감자(감자),20kg,25000,강원 홍천군,normal,2021-10-30,8
509827,[감자]감자(감자),20kg,24000,강원 홍천군,normal,2021-10-30,8


### price

In [61]:
# 계산을 위해 scale 값 미리 변경
# 현재 값들은 string이므로 float 형태로 변환할 필요가 있음
# 몇몇 이상치 처리도 진행
def scale_checker(scale):
  if scale.startswith('.')==True:
    return format(float('0'+scale[:-2]),'.2f')
  else:
    return format(float((scale[:-2])),'.2f')
df2['scale'] = df2['scale'].apply(scale_checker)
df2['scale'] = df2['scale'].astype('float')
# df2['scale'] = format(round(df2['scale'].apply(pd.to_numeric),2),'.2f')

In [62]:
df2['scale'].unique()

array([     5.  ,      4.  ,     16.  ,     15.  ,     10.  ,     17.  ,
           19.  ,     14.  ,     18.  ,      8.  ,      7.  ,     11.  ,
           13.  ,      6.  ,     12.  ,      9.  ,      2.  ,      1.5 ,
            4.5 ,      2.5 ,      3.  ,      0.5 ,      0.2 ,      1.  ,
           20.  ,      4.8 ,      1.8 ,      1.2 ,      0.4 ,      3.6 ,
            3.5 ,      0.3 ,      0.09,      6.5 ,      7.5 ,      0.1 ,
           21.  ,    211.  ,     10.01,     20.01,      0.7 ,      0.8 ,
           11.5 ,      2.01,      4.01,      2.6 ,      3.8 ,      8.6 ,
            8.01,      9.6 ,     10.4 ,     25.  ,      5.1 ,      2.3 ,
            2.4 ,      0.25,      0.15,      0.06,      0.05,      2.7 ,
            3.4 ,      2.9 ,      4.6 ,      3.1 ,      4.9 ,      2.8 ,
            5.8 ,     14.5 ,     13.5 ,      5.2 ,      0.12,      0.6 ,
           22.  ,      5.5 ,      4.2 ,      5.3 ,      3.2 ,    500.  ,
            5.6 ,     24.  ,     18.01,     10.5 , 

In [63]:
# 1원짜리 거래 발견, 너무 거래값이 작은 값들은 삭제
pd.options.display.float_format = '{:.5f}'.format
df2['price'].describe()

count     509829.00000
mean       23970.24567
std       137487.92397
min            5.00000
25%         8500.00000
50%        15000.00000
75%        26400.00000
max     60507000.00000
Name: price, dtype: float64

In [64]:
df2 = df2.drop(df2[df2['price']<500].index)

In [65]:
df2

,prd,scale,price,origin,eco,reg_date,new_class
0,[버섯]팽이버섯,5.00000,5500,경북 청도군,ecofriendly,2021-09-30,1
1,[버섯]팽이버섯,5.00000,4000,전남 무안군,ecofriendly,2021-09-30,1
2,[버섯]팽이버섯,5.00000,10000,경북 청도군,good_prd,2021-09-30,1
3,[버섯]팽이버섯,5.00000,8000,경북 청도군,good_prd,2021-09-30,1
4,[버섯]팽이버섯,5.00000,9500,경북 청도군,good_prd,2021-09-30,1
...,...,...,...,...,...,...,...
509824,[호박]늙은호박,10.00000,800,전남 해남군,normal,2021-10-29,8
509825,[감자]감자(감자),20.00000,28000,강원 홍천군,normal,2021-10-30,8
509826,[감자]감자(감자),20.00000,25000,강원 홍천군,normal,2021-10-30,8
509827,[감자]감자(감자),20.00000,24000,강원 홍천군,normal,2021-10-30,8


In [66]:
# kg 당 가격 생성

df2['price_kg'] = round((df2['price']/df2['scale']),2)

### prd
- 품목명 정리
- 품목 종류 정리

In [67]:
# 대부분 [상품 종류]상품명 으로 정리되어 있는걸 알 수 있음
df2['prd'].value_counts()

[오이]백다다기                11668
[호박]애호박                 10245
[깻잎]깻잎                   7854
[시금치]시금치                 6253
[가지]가지                   5724
                        ...  
[빨간양배추(루비볼)]빨간양배추(일반        1
[마늘]깐마늘 한지                  1
[감귤]한라봉                     1
[기타양채]적오크림                  1
[사과]사과(콘티)                  1
Name: prd, Length: 1444, dtype: int64

In [68]:
# 정리를 위해 상품명 삭제
def prd_sort(words):
  p = re.compile('[ㄱ-힣]+')
  word = re.search(p,words)
  return word.group()

df2['prd'] = df2['prd'].apply(prd_sort)

In [69]:
# 기타 품목 삭제
df2 = df2.drop(df2[df2['prd']=='기타'].index)

In [70]:
# 국산,수입산,깐,기타 삭제
def prd_sort2(words):
  p = re.compile('[^국산수입깐기타]+')
  word = re.search(p,str(words))
  return word.group()

df2["prd"] = df2["prd"].apply(prd_sort2)


In [71]:
# 몇몇 품목들 정리가 필요
a = sorted(df2['prd'].unique())
for i in range(len(a)):
  if i%15 != 0:
    print(a[i],end=' ')
  else:
    print(a[i],end=' ')
    print()

가리비 
가물치 가오리 가오리채 가자미 가지 각굴 갈치 감 감귤 감말랭이 감자 갑오징어 갑오징어몸통 갑오징어살 갓 
강낭콩 개조개 건고구마순 건고추 게 게지 겨자 고 고구마 고구마순 고둥 고들빼 고등어 고추 고추잎 
곤달비 곤드레나물 골뱅이 곶감 과실류 과일 과일류 과채 과채류 굴 귤 그레이프푸룻 근대 근채류 깻잎 
꼬막 꼴뚜 꽁치 꽈리고추 낙지 냉이 논우렁이 눈볼대 느 다래 단감 달래 닭새우 당귀 당근 
대구 대구고니 대구머리 대구포 대추 대파 더덕 덟은감 도다리 도라지 도토리 돗나물 동백하새우 동부콩 동자개 
동죽 두류 두리안 딸 땅콩 떫은감 라임 람부탄 레몬 리치 마 마늘 만가닥 맛 망고 
매생이 머위대 메 메론 명태 명태곤니 명태알 명태포 모과 모시조개 무 무순 무우 무청 무화과 
문어 문어바지락 물가자미 미꾸라지 미나리 미더덕 민들레 민물새우 민어 바나나 바지락 박 밤 방울토마토 방풍 
방풍나물 배 배추 백새우 백합 뱀장어 버섯 버섯류 병어 복숭아 복어 봄동배추 봉지굴 부세 부지갱이 
부추 붉은고추 붕어 붕장어 브로커리 브로코리 블루베리 비단멍게 비름 비트 빈스 빨간양배추 삐틀이 사과 삼 
삼치 상추 새꼬막 새송이 새싹 새우 새우살 새조개 샐러리 생강 생고추 석류 선인장열매 세러리 셀러리 
소라 속새 숙주나물 순무 숫꽃게 시금치 실파 쌈채류 쑥 쑥갓 아귀 아보카도 아보카드 아스파라가스 아스파라거스 
아욱 알 알배 암꽃게 야자 야콘 양배추 양상추 양송이 양채 양채류 양파 얼갈이 얼갈이배추 여주 
연근 연어 연어살 열무 염고등어 엽경채류 엽채 엽채류 오렌지 오만둥이 오이 오징어 오징어몸통 오징어원양 오징어채 
옥 완두콩 용과 우렁 우렁쉥이 우렁이 유채 으름 은어 은행 임연 잉어 자두 자몽 재첩 
적근대 적새우 적채 전복 전어 절단 절단낙지 젓새우 중하 쪽파 쭈꾸미 참나물 참다래 참외 참조 
채소 청경채 청어 총각무 취나물 치커리 칠게 칼라후라워 칼리플라워 칼리후라워 케일 코다리명태 코라비 코코넛 콜라비 
콩 콩나물 키위 키조개 키조개날개 토란 토란대 토마

In [72]:
### 예시
# df2[(df2['prd']=='유채') & (df2['price']==4500)]['prd'].unique()
# df2[df2['prd']=='유채']['prd'].unique()

In [73]:
### 원래 단어 뭐였는지 검색
# org_df[org_df['품목명'].str.contains('느')]['품목명'].unique()

In [74]:
### 현재 이름 검색
# df2[df2['prd']=='느']

In [75]:
###
# JG

# 고 -> 고수로 변경
df2.loc[df2["prd"]=="고","prd"] = "고수"
# 과실류, 과일류, 과채, 채소 -> 과채류로 통합
df2.loc[df2["prd"]=="과실류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일야채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일과채류","prd"] = "과채류"
df2.loc[df2["prd"]=="과일","prd"] = "과채류"
df2.loc[df2["prd"]=="과채","prd"] = "과채류"
df2.loc[df2["prd"]=="채소","prd"] = "과채류"
# 고들빼 -> 고들빼기로 변경
df2.loc[df2["prd"]=="고들빼","prd"] = "고들빼기"
# 꼴뚜 -> 꼴두기로 변경
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
# 그레이프푸룻 -> 자몽으로 통합
df2.loc[df2["prd"]=="그레이프푸룻","prd"] = "자몽"

# 느 -> 느타리버섯으로 통합
df2.loc[df2["prd"]=="느","prd"] = "느타리버섯"

# 대구 관련 품목들 정리
df2.loc[df2["prd"]=="대구고니","prd"] = "대구"
df2.loc[df2["prd"]=="대구머리","prd"] = "대구"
df2.loc[df2["prd"]=="대구포","prd"] = "대구"
# 듀리안 -> 두리안으로 통합
df2.loc[df2["prd"]=="듀리안","prd"] = "두리안"
# 딸 -> 딸기로 변경
df2.loc[df2["prd"]=="딸","prd"] = "딸기"

# 만가닥 -> 만가닥버섯으로 변경
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
# 맛 -> 맛조개로 변경
df2.loc[df2["prd"]=="맛","prd"] = "맛조개"
# 망고스턴 -> 망고스틴으로 수정
df2.loc[df2["prd"]=="망고스턴","prd"] = "망고스틴"
# 메 -> 메기로 변경
df2.loc[df2["prd"]=="메","prd"] = "메기"
# 명태 관련 품목들 정리
df2.loc[df2["prd"]=="명태곤니","prd"] = "명태"
df2.loc[df2["prd"]=="명태알","prd"] = "명태"
df2.loc[df2["prd"]=="명태포","prd"] = "명태"
df2.loc[df2["prd"]=="코다리명태","prd"] = "명태"
# 무우 -> 무로 통합
df2.loc[df2["prd"]=="무우","prd"] = "무"

# 방풍 -> 방풍나물로 통합
df2.loc[df2["prd"]=="방풍","prd"] = "방풍나물"
# 버섯류 -> 버섯으로 변경
df2.loc[df2["prd"]=="버섯","prd"] = "버섯"
# 벗,버찌 -> 체리로 통합
df2.loc[df2["prd"]=="벗","prd"] = "체리"
df2.loc[df2["prd"]=="버찌","prd"] = "체리"
df2.loc[df2["prd"]=="벗찌","prd"] = "체리"
# 브로커리, 브로코리 -> 브로콜리로 통합
df2.loc[df2["prd"]=="브로커리","prd"] = "브로콜리"
df2.loc[df2["prd"]=="브로코리","prd"] = "브로콜리"

# 삼 -> 수삼으로 변경
df2.loc[df2["prd"]=="삼","prd"] = "수삼"
# 새우살 -> 새우로 변경
df2.loc[df2["prd"]=="새우살","prd"] = "새우"
# 세러리, 셀러리 -> 샐러리로 통합
df2.loc[df2["prd"]=="세러리","prd"] = "샐러리"
df2.loc[df2["prd"]=="셀러리","prd"] = "샐러리"
# 생고추 -> 고추로 통합
df2.loc[df2["prd"]=="생고추","prd"] = "고추"

# 아보카드 -> 아보카도로 변경
df2.loc[df2["prd"]=="아보카드","prd"] = "아보카도"
# 아스파라가스 -> 아스파라거스로 변경
df2.loc[df2["prd"]=="아스파라가스","prd"] = "아스파라거스"
# 알 -> 알타리무로 변경
df2.loc[df2["prd"]=="알","prd"] = "알타리무"
# 알배 -> 알배기로 변경
df2.loc[df2["prd"]=="알배","prd"] = "알배기"
# 양채 -> 양채류로 통합
df2.loc[df2["prd"]=="양채","prd"] = "양채류"
# 얼갈이 -> 얼갈이배추로 통합
df2.loc[df2["prd"]=="얼갈이","prd"] = "얼갈이배추"
# 엽채류, 엽채 -> 엽경채류로 통합
df2.loc[df2["prd"]=="엽채류","prd"] = "엽경채류"
df2.loc[df2["prd"]=="엽채","prd"] = "엽경채류"
# 오징어 관련 품목들 정리
df2.loc[df2["prd"]=="오징어다리","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어원양","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채","prd"] = "오징어"
df2.loc[df2["prd"]=="활복오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어채원양","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어살","prd"] = "오징어"
df2.loc[df2["prd"]=="갑오징어몸통","prd"] = "오징어"
# 옥, 풋옥 -> 옥수수로 정리
df2.loc[df2["prd"]=="옥","prd"] = "옥수수"
df2.loc[df2["prd"]=="풋옥","prd"] = "옥수수"
# 임연 -> 임연수어로 정리
df2.loc[df2["prd"]=="임연","prd"] = "임연수어"


# 절단 -> 절단쭈꾸미 -> 쭈꾸미로 정리
df2.loc[df2["prd"]=="절단","prd"] = "쭈꾸미"
# 조개살 -> 조개로 정리
df2.loc[df2["prd"]=="조개살","prd"] = "조개"

# 참조 -> 참조기로 정리
df2.loc[df2["prd"]=="참조","prd"] = "참조기"

# 칼라후라워,칼리플라워, 칼리후라워 -> 컬리플라워로 통합
df2.loc[df2["prd"]=="칼라후라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리플라워","prd"] = "컬리플라워"
df2.loc[df2["prd"]=="칼리후라워","prd"] = "컬리플라워"
# 코라비 -> 콜라비로 통합
df2.loc[df2["prd"]=="코라비","prd"] = "콜라비"
# 키조개날개 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개날개","prd"] = "키조개"

# 팽이 -> 팽이버섯으로 통합
df2.loc[df2["prd"]=="팽이","prd"] = "팽이버섯"
# 파세리 -> 파슬리로 변경
df2.loc[df2["prd"]=="파세리","prd"] = "파슬리"
# 표고 -> 표고버섯으로 통합
df2.loc[df2["prd"]=="표고","prd"] = "표고버섯"
# 포장바지락 -> 바지락으로 변경
df2.loc[df2["prd"]=="포장바지락","prd"] = "바지락"

### 일단 여기까지 실행
# JY
#각굴 -> 굴로 통합
df2.loc[df2["prd"]=="각굴","prd"] = "굴"
#가오리채 -> 가오리로 통합
df2.loc[df2["prd"]=="가오리채","prd"] = "가오리"
# 게지 -> 키조개로 통합
df2.loc[df2["prd"]=="게지","prd"] = "키조개"
#고 -> 고구마로 통합
df2.loc[df2["prd"]=="고","prd"] = "고구마"
#꼴뚜 -> 꼴뚜기로 통합
df2.loc[df2["prd"]=="꼴뚜","prd"] = "꼴뚜기"
#돗나물 -> 돌나물로 통합
df2.loc[df2["prd"]=="돗나물","prd"] = "돌나물"
#레드쉬 -> 레디쉬로 통합
df2.loc[df2["prd"]=="레드쉬","prd"] = "레디쉬"
#만가닥 -> 만가닥버섯으로 통합
df2.loc[df2["prd"]=="만가닥","prd"] = "만가닥버섯"
#메 -> 메기로 통합
df2.loc[df2["prd"]=="메","prd"] = "메기"
#봄동 -> 봄동배추로 통합
df2.loc[df2["prd"]=="봄동","prd"] = "봄동배추"
#봉지굴 -> 굴로 통합
df2.loc[df2["prd"]=="봉지굴","prd"] = "굴"
#박 -> 수박으로 통합
df2.loc[df2["prd"]=="박","prd"] = "수박"
#백조 -> 조기로 통합 
df2.loc[df2["prd"]=="백조","prd"] = "조기"
#벅굴 -> 벚굴로 통합
df2.loc[df2["prd"]=="벅굴","prd"] = "벚굴"
#비 -> 비타민으로 통합
df2.loc[df2["prd"]=="비","prd"] = "비타민"
#빈스 -> 콩으로 통합
df2.loc[df2["prd"]=="빈스","prd"] = "콩"
#삐틀이 -> 고둥으로 통합
df2.loc[df2["prd"]=="삐틀이","prd"] = "고둥"
#새송이 -> 새송이버섯으로 통합
df2.loc[df2["prd"]=="새송이","prd"] = "새송이버섯"
#세트 -> 과채류로 통합
df2.loc[df2["prd"]=="세트","prd"] = "과채류"
#알 -> 알로애로 통합
df2.loc[df2["prd"]=="알","prd"] = "알로에"
#엄나무 -> 음나무로 통합
df2.loc[df2["prd"]=="엄나무","prd"] = "음나무"
#염고등어 -> 고등어로 통합
df2.loc[df2["prd"]=="염고등어","prd"] = "고등어"
#오만둥이 -> 미더덕으로 통합
df2.loc[df2["prd"]=="오만둥이","prd"] = "미더덕"
#오징어 관련 품목들 정리
df2.loc[df2["prd"]=="선동오징어","prd"] = "오징어"
df2.loc[df2["prd"]=="오징어몸통","prd"] = "오징어"
#우렁쉥이 -> 멍게로 통합
df2.loc[df2["prd"]=="우렁쉥이","prd"] = "멍게"
#우뭇가시리 -> 우뭇가사리로 통합
df2.loc[df2["prd"]=="우뭇가시리","prd"] = "우뭇가사리"
#적채 -> 적양배추로 통합
df2.loc[df2["prd"]=="적채","prd"] = "적양배추"
#절단낙지 -> 낙지로 통합
df2.loc[df2["prd"]=="절단낙지","prd"] = "낙지"
#조 -> 조개로 통합
df2.loc[df2["prd"]=="조","prd"] = "조개"
#청어원양 -> 청어로 통합
df2.loc[df2["prd"]=="청어원양","prd"] = "청어"
#키조개꼭지 -> 키조개로 통합
df2.loc[df2["prd"]=="키조개꼭지","prd"] = "키조개"

# 암꽃게, 숫꽃게 -> 꽃게로 통합
df2.loc[df2["prd"]=="암꽃게","prd"] = "꽃게"
df2.loc[df2["prd"]=="숫꽃게","prd"] = "꽃게"
# 우렁 -> 우렁이로 통합
df2.loc[df2["prd"]=="우렁","prd"] = "우렁이"
# 알로애 -> 알로에로 변경
df2.loc[df2["prd"]=="알로애","prd"] = "알로에"
# 피마자잎 -> 피마자로 변경
df2.loc[df2["prd"]=="피미자잎","prd"] = "피마자"
# 식용허브 -> 허브로 변경
df2.loc[df2["prd"]=="식용허브","prd"] = "허브"
# 선인장열매 -> 선인장으로 변경
df2.loc[df2["prd"]=="선인장열매","prd"] = "선인장"
# 쌈추 -> 쌈채류로 변경
df2.loc[df2["prd"]=="쌈추","prd"] = "쌈채류"
# 덟은감 -> 감으로 변경
df2.loc[df2["prd"]=="덟은감","prd"] = "감"
# 피뿔고동 -> 고둥으로 변경
df2.loc[df2["prd"]=="피뿔고동","prd"] = "고둥"
# 꼬시래 -> 꼬시래기으로 변경
df2.loc[df2["prd"]=="꼬시래","prd"] = "꼬시래기"
# 연어살 -> 연어로 변경
df2.loc[df2["prd"]=="연어살","prd"] = "연어"
# 비단멍게 -> 멍게로 변경
df2.loc[df2["prd"]=="비단멍게","prd"] = "멍게"
# 물가자미 -> 가자미로 변경
df2.loc[df2["prd"]=="물가자미","prd"] = "가자미"
# 조미제품 삭제
df2.drop(df2[df2['prd']=='조미제품'].index,inplace=True)
df2.drop(df2[df2['prd']=='어류원양'].index,inplace=True)

In [76]:
# 기존 products 리스트와 비교
ex_prd = pd.read_csv("../products.csv")
df2[~df2['prd'].isin(ex_prd['prd'])]['prd'].unique()

array([], dtype=object)

In [77]:
# 물품 리스트 최신화
new_prd = pd.DataFrame(df2['prd'].unique(),columns=['prd'])
# products = ex_prd.append(new_prd[~new_prd['prd'].isin(ex_prd['prd'])],ignore_index=True)
products = ex_prd.append(new_prd,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_2444/2619460125.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  products = ex_prd.append(new_prd,ignore_index=True)


In [78]:
products.drop(columns=['Unnamed: 0'],inplace=True)
products.drop_duplicates(inplace=True)
products.reset_index(inplace=True,drop=True)

In [79]:
# 다음 작업을 위해 저장
products.to_csv('../products.csv')

In [80]:
# 전체 거래량과 품목량
# 비교를 위해 분포 파악
# 거래량이 너무 적은 품목은 삭제할 필요성이 있음

prd_lst = (df2['prd'].value_counts()).to_frame()
prd_lst.describe()

,prd
count,234.00000
mean,2175.61111
std,5361.22014
min,1.00000
25%,15.25000
50%,114.50000
75%,1216.50000
max,36914.00000


In [81]:
# 50개 미만 품목은 비교가 힘드므로 삭제
# 줄어든 종류만 남기기

prd_lst = prd_lst[prd_lst['prd']>=50]
prd_lst.reset_index(inplace=True)
prd_lst = prd_lst.rename(columns = {'index':'prd','prd':'count'})

In [82]:
df3 = df2.merge(prd_lst)
df3[df3['count']>=50]['prd'].value_counts()

버섯     36914
호박     26488
고추     25941
감      24258
오이     22781
       ...  
동죽        56
곶감        56
도라지       54
건고추       52
야콘        50
Name: prd, Length: 145, dtype: int64

In [83]:
# 줄어든 것 확인했으므로 count column은 삭제
df3 = df3.drop(columns='count')

### scale

In [84]:
np.set_printoptions(precision=2, suppress=True)
df3['scale'].unique()

array([    5.  ,     4.  ,    16.  ,    15.  ,    10.  ,    17.  ,
          19.  ,    14.  ,    18.  ,     8.  ,     7.  ,    11.  ,
          13.  ,     6.  ,    12.  ,     9.  ,     2.  ,     1.5 ,
           1.  ,     0.12,     0.3 ,     0.5 ,     0.6 ,     3.  ,
           2.5 ,    20.  ,    21.  ,     0.03,     0.2 ,     1.2 ,
           1.8 ,     0.15,     0.17,    24.  ,    22.  ,     4.5 ,
          23.  ,     3.5 ,     0.56,     8.5 ,     3.6 ,     6.5 ,
           7.5 ,     3.72,     2.6 ,    14.4 ,     9.5 ,     2.7 ,
           2.8 ,     3.8 ,     0.25,     5.5 ,     2.3 ,     4.8 ,
           0.7 ,     9.6 ,     1.6 ,     0.4 ,    26.  ,    25.  ,
           2.4 ,     4.2 ,     5.3 ,     3.3 ,     3.4 ,     0.1 ,
           3.7 ,     6.1 ,     3.62,    11.6 ,     4.6 ,     4.7 ,
           3.1 ,     7.6 ,     2.9 ,    11.9 ,    10.2 ,     3.9 ,
           9.3 ,     3.2 ,    11.5 ,    12.5 ,     5.1 ,    10.9 ,
           2.1 ,     2.2 ,    13.5 ,     7.4 ,     5.7 ,     5

In [87]:
# 50kg 이상 거래된 건들 이상치라 판단하고 확인 진행
df3[df3['scale']>=50]['prd'].unique()

array(['오이', '토마토', '호박', '대추', '무', '미나리', '바지락'], dtype=object)

In [111]:
# 스케일 오표기된 항목 수정
# 이상치 항목 삭제
df3 = df3.drop(df3[(df3['scale']>=50)].index)

In [112]:
df3[df3['scale']>=50]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg


### origin

In [113]:
# 도,시, 국산, 수입 표기 분류 필요
df3['origin'].unique()

array(['경북 청도군', '전남 무안군', '전남 나주시', '전남 함평군', '수입산 중국', '충남 홍성군',
       '충남 예산군', '경기 화성시', '충북 옥천군', '경기 평택시', '충남 당진시', '경기 용인시',
       '충북 충주시', '충북 괴산군', '전남 구례군', '경기 여주시', '충남 청양군', '충북 음성군',
       '경기 의왕시', '전남 해남군', '경기 양평군', '충남 보령시', '강원 홍천군', '경기 안성시',
       '충남 아산시', '강원 인제군', '강원 춘천시', '충남 서산시', '강원 양양군', '강원 영월군',
       '경기 광주시', '경기 이천시', '세종시', '충남 부여군', '경북 상주시', '충남 천안시', '충북 청주시',
       '경북 성주군', '전남 담양군', '경북 문경시', '경북 김천시', '경남 함양군', '전남 여수시',
       '청주시 상당구', '충북 청원군', '청주시 흥덕구', '전북 정읍시', '중국', '강원 평창군', '경기 성남시',
       '경기 파주시', '전북 고창군', '전남 장흥군', '충남 논산시', '전남 보성군', '경기 포천시',
       '충남 서천군', '제주 제주시', '인천 강화군', '세종', '강원 화천군', '충남 공주시', '경남 산청군',
       '전북 부안군', '경남 의령군', '전남 장성군', '충북 영동군', '전북 군산시', '경기 의정부시',
       '충북 보은군', '전남 고흥군', '경기 가평군', '강원 철원군', '경기 양주시', '경북 예천군', '기타외국',
       '경남 진주시', '경남 창녕군', '충북 증평군', '강원 횡성군', '충북 진천군', '경기 고양시',
       '강원 원주시', '경기 남양주시', '경기 연천군', '충남 금산군', '전북 김제시', '강원 고성군',
       '경기 김포시', '경북 의성군', '충북 

In [114]:
# 해당 조건에 맞는 값 추출

splits = df3['origin'].str.split(' ')
cacul = splits.apply(lambda x: pd.Series(x))
df66 = pd.merge(df3,cacul, left_index=True, right_index=True, how='inner')
df66

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,0,1,2,3
0,버섯,5.00000,5500,경북 청도군,ecofriendly,2021-09-30,1,1100.00000,경북,청도군,NaN,NaN
1,버섯,5.00000,4000,전남 무안군,ecofriendly,2021-09-30,1,800.00000,전남,무안군,NaN,NaN
2,버섯,5.00000,10000,경북 청도군,good_prd,2021-09-30,1,2000.00000,경북,청도군,NaN,NaN
3,버섯,5.00000,8000,경북 청도군,good_prd,2021-09-30,1,1600.00000,경북,청도군,NaN,NaN
4,버섯,5.00000,9500,경북 청도군,good_prd,2021-09-30,1,1900.00000,경북,청도군,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
507900,동백하새우,8.00000,145000,국내산,normal,2021-10-30,3,18125.00000,국내산,NaN,NaN,NaN
507901,동백하새우,8.00000,165000,국내산,normal,2021-10-30,3,20625.00000,국내산,NaN,NaN,NaN
507902,동백하새우,8.00000,100000,국내산,normal,2021-10-30,3,12500.00000,국내산,NaN,NaN,NaN
507903,동백하새우,8.00000,170000,국내산,normal,2021-10-30,3,21250.00000,국내산,NaN,NaN,NaN


In [115]:
# 세부 카테고리 정리 (state, city 편집 시 여기서 시작)

df4 = df66.rename(columns = {0:'state',1:'city'})

# 2,3은 필요없는 정보이므로 삭제
df4 = df4.drop(columns=[2,3])

In [116]:
df4['state'].unique()

array(['경북', '전남', '수입산', '충남', '경기', '충북', '강원', '세종시', '경남', '청주시',
       '전북', '중국', '제주', '인천', '세종', '기타외국', '대구', '경기도', '서울', '서울시',
       '성남시', '광주', '칠레', '미국', '필리핀', '페루', '강원도', '서울특별시', '인천시',
       '전라북도', '대전시', '대전', '포항시', '부산시', '호주', '경북/문경시', '경북/상주시',
       '오스트레일', '남아프리카', '남아프리카공화국', '오스트레일리아', '베트남', '제주도', '뉴질랜드',
       '제주특별자치도', '타이', '인천광역시', '광주시', '부산', '일본', '부산광역시', '전주시', '멕시코',
       '우즈베키스탄', '태국', '브라질', '부천시', '에콰도르', '코스타리카', '콜롬비아', '에쿠아도르',
       '에콰돌', '인도네시아', '과테말라', '우크라이나', '세종특별자치시', '충북청주시상당구', '국내산',
       '수입', '포클랜드'], dtype=object)

In [117]:
# 시,도 제거
def name_sort(words):
  p = re.compile('[^시도]+')
  word = re.search(p,str(words))
  return word.group()

df4["state"] = df4["state"].apply(name_sort)

# 오타도 제거

def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["state"] = df4["state"].apply(name_sort2)

In [118]:
df4["state"].unique()

array(['경북', '전남', '수입산', '충남', '경기', '충북', '강원', '세종', '경남', '청주', '전북',
       '중국', '제주', '인천', '기타외국', '대구', '서울', '성남', '광주', '칠레', '미국',
       '필리핀', '페루', '서울특별', '전라북', '대전', '포항', '부산', '호주', '경북/문경',
       '경북/상주', '오스트레일', '남아프리카', '남아프리카공화국', '오스트레일리아', '베트남', '뉴질랜드',
       '제주특별자치', '타이', '인천광역', '일본', '부산광역', '전주', '멕', '우즈베키스탄', '태국',
       '브라질', '부천', '에콰', '코스타리카', '콜롬비아', '에쿠아', '에콰돌', '인', '과테말라',
       '우크라이나', '세종특별자치', '충북청주', '국내산', '수입', '포클랜드'], dtype=object)

In [119]:
# state
# 특이케이스 제거

# 국내
df4.loc[df4["state"]=="국내산","city"] = "미상"
df4.loc[df4["state"]=="국내산","state"] = "국산"
df4.loc[df4["state"]=="원양산","city"] = "미상"

## 도단위
df4.loc[df4["state"]=="충청북","state"] = "충북"
df4.loc[df4["state"]=="충청남","state"] = "충남"
df4.loc[df4["state"]=="전라북","state"] = "전북"
df4.loc[df4["state"]=="전라남","state"] = "전남"
df4.loc[df4["state"]=="경상북","state"] = "경북"
df4.loc[df4["state"]=="경상남","state"] = "경남"

## 시단위
### 광역시

df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="대구","city"] = "대구"
df4.loc[df4["state"]=="서울특별","city"] = "서울"
df4.loc[df4["state"]=="서울특별","state"] = "서울"
df4.loc[df4["state"]=="인천광역","city"] = "인천"
df4.loc[df4["state"]=="인천광역","state"] = "인천"
df4.loc[df4["state"]=="부산","city"] = "부산"
df4.loc[df4["state"]=="부산광역","city"] = "부산"
df4.loc[df4["state"]=="부산광역","state"] = "부산"
df4.loc[df4["state"]=="가락동","city"] = "서울"
df4.loc[df4["state"]=="가락동","state"] = "서울"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","city"] = "세종"
df4.loc[df4["state"]=="세종특별자치","state"] = "세종"
df4.loc[df4["state"]=="대전","city"] = "대전"
df4.loc[df4["state"]=="제주","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","city"] = "제주"
df4.loc[df4["state"]=="제주특별자치","state"] = "제주"
df4.loc[df4["state"]=="제주특별자","city"] = "제주"
df4.loc[df4["state"]=="제주특별자","state"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","city"] = "제주"
df4.loc[df4["state"]=="제주/서귀포","state"] = "제주"
df4.loc[df4["state"]=="세종","city"] = "세종"
df4.loc[df4["state"]=="울산","city"] = "울산"
df4.loc[df4["state"]=="울산","state"] = "울산"

### 일반시
df4.loc[df4["state"]=="성남","city"] = "성남"
df4.loc[df4["state"]=="성남","state"] = "경기"
df4.loc[df4["state"]=="전주","city"] = "전주"
df4.loc[df4["state"]=="전주","state"] = "전북"
df4.loc[df4["state"]=="청주","city"] = "청주"
df4.loc[df4["state"]=="청주","state"] = "충북"
df4.loc[df4["state"]=="수원","city"] = "수원"
df4.loc[df4["state"]=="수원","state"] = "경기"
df4.loc[df4["state"]=="부천","city"] = "부천"
df4.loc[df4["state"]=="부천","state"] = "경기"
df4.loc[df4["state"]=="경기고양","city"] = "고양"
df4.loc[df4["state"]=="경기고양","state"] = "경기"
df4.loc[df4["state"]=="충북청주","city"] = "청주"
df4.loc[df4["state"]=="충북청주","state"] = "충북"
df4.loc[df4["state"]=="전북전주","city"] = "전주"
df4.loc[df4["state"]=="전북전주","state"] = "전북"
df4.loc[df4["state"]=="경북/문경","city"] = "문경"
df4.loc[df4["state"]=="경북/문경","state"] = "경북"
df4.loc[df4["state"]=="포항","city"] = "포항"
df4.loc[df4["state"]=="포항","state"] = "경북"
df4.loc[df4["state"]=="경북/안동","city"] = "안동"
df4.loc[df4["state"]=="경북/안동","state"] = "경북"
df4.loc[df4["state"]=="전남/무안군","city"] = "전남"
df4.loc[df4["state"]=="전남/무안군","state"] = "무안"
df4.loc[df4["state"]=="경북/구미","city"] = "구미"
df4.loc[df4["state"]=="경북/구미","state"] = "경북"
df4.loc[df4["state"]=="경북/상주","city"] = "상주"
df4.loc[df4["state"]=="경북/상주","state"] = "경북"
df4.loc[df4["state"]=="경북/영천","city"] = "영천"
df4.loc[df4["state"]=="경북/영천","state"] = "경북"
df4.loc[df4["state"]=="경남/합천군","city"] = "합천"
df4.loc[df4["state"]=="경남/합천군","state"] = "경남"
df4.loc[df4["state"]=="무안","state"] = "전남"
df4.loc[df4["state"]=="서해안","city"] = "미상"
df4.loc[df4["state"]=="서해안","state"] = "국산"


# 외국
## 아시아
df4.loc[df4["state"]=="칠레","city"] = "칠레"
df4.loc[df4["state"]=="칠레","state"] = "수입"
df4.loc[df4["state"]=="일본","city"] = "일본"
df4.loc[df4["state"]=="일본","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="우즈베크","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베크","state"] = "수입"
df4.loc[df4["state"]=="이스라엘","city"] = "이스라엘"
df4.loc[df4["state"]=="이스라엘","state"] = "수입"
df4.loc[df4["state"]=="터키","city"] = "터키"
df4.loc[df4["state"]=="터키","state"] = "수입"
df4.loc[df4["state"]=="중국","city"] = "중국"
df4.loc[df4["state"]=="중국","state"] = "수입"
df4.loc[df4["state"]=="타이","city"] = "태국"
df4.loc[df4["state"]=="타이","state"] = "수입"
df4.loc[df4["state"]=="태국","city"] = "태국"
df4.loc[df4["state"]=="태국","state"] = "수입"
df4.loc[df4["state"]=="필리핀","city"] = "필리핀"
df4.loc[df4["state"]=="필리핀","state"] = "수입"
df4.loc[df4["state"]=="우즈베키스탄","city"] = "우즈베키스탄"
df4.loc[df4["state"]=="우즈베키스탄","state"] = "수입"
df4.loc[df4["state"]=="대만","city"] = "대만"
df4.loc[df4["state"]=="대만","state"] = "수입"
df4.loc[df4["state"]=="베트남","city"] = "베트남"
df4.loc[df4["state"]=="베트남","state"] = "수입"
df4.loc[df4["state"]=="이란","city"] = "이란"
df4.loc[df4["state"]=="이란","state"] = "수입"
df4.loc[df4["state"]=="러","city"] = "러시아"
df4.loc[df4["state"]=="러","state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"city"] = "인도네시아"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도네시아"),"state"] = "수입"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"city"] = "인도"
df4.loc[(df4["state"]=="인")&(df4["origin"]=="인도"),"state"] = "수입"

## 아메리카

df4.loc[df4["state"]=="캘리포니아","city"] = "미국"
df4.loc[df4["state"]=="캘리포니아","state"] = "수입"
df4.loc[df4["state"]=="포클랜드","city"] = "영국"
df4.loc[df4["state"]=="포클랜드","state"] = "수입"
df4.loc[df4["state"]=="콜롬비아","city"] = "콜롬비아"
df4.loc[df4["state"]=="콜롬비아","state"] = "수입"
df4.loc[df4["state"]=="에콰","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰","state"] = "수입"
df4.loc[df4["state"]=="에쿠아","city"] = "에콰도르"
df4.loc[df4["state"]=="에쿠아","state"] = "수입"
df4.loc[df4["state"]=="에콰돌","city"] = "에콰도르"
df4.loc[df4["state"]=="에콰돌","state"] = "수입"
df4.loc[df4["state"]=="과테말라","city"] = "과테말라"
df4.loc[df4["state"]=="과테말라","state"] = "수입"
df4.loc[df4["state"]=="코스타리카","city"] = "코스타리카"
df4.loc[df4["state"]=="코스타리카","state"] = "수입"
df4.loc[df4["state"]=="멕","city"] = "멕시코"
df4.loc[df4["state"]=="멕","state"] = "수입"
df4.loc[df4["state"]=="페루","city"] = "페루"
df4.loc[df4["state"]=="페루","state"] = "수입"
df4.loc[df4["state"]=="미국","city"] = "미국"
df4.loc[df4["state"]=="미국","state"] = "수입"
df4.loc[df4["state"]=="아르헨티나","city"] = "아르헨티나"
df4.loc[df4["state"]=="아르헨티나","state"] = "수입"
df4.loc[df4["state"]=="브라질","city"] = "브라질"
df4.loc[df4["state"]=="브라질","state"] = "수입"

## 유럽
df4.loc[df4["state"]=="스페인","city"] = "스페인"
df4.loc[df4["state"]=="스페인","state"] = "수입"
df4.loc[df4["state"]=="우크라이나","city"] = "우크라이나"
df4.loc[df4["state"]=="우크라이나","state"] = "수입"
df4.loc[df4["state"]=="이탈리아","city"] = "이탈리아"
df4.loc[df4["state"]=="이탈리아","state"] = "수입"
df4.loc[df4["state"]=="불가리아","city"] = "불가리아"
df4.loc[df4["state"]=="불가리아","state"] = "수입"

## 아프리카
df4.loc[df4["state"]=="남아프리카","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카","state"] = "수입"
df4.loc[df4["state"]=="남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="남아프리카공화국","state"] = "수입"
df4.loc[df4["state"]=="수입남아프리카공화국","city"] = "남아공"
df4.loc[df4["state"]=="수입남아프리카공화국","state"] = "수입"


## 오세아니아
df4.loc[df4["state"]=="뉴질랜드","city"] = "뉴질랜드"
df4.loc[df4["state"]=="뉴질랜드","state"] = "수입"
df4.loc[df4["state"]=="오스트레일","city"] = "호주"
df4.loc[df4["state"]=="오스트레일","state"] = "수입"
df4.loc[df4["state"]=="오스트레일리아","city"] = "호주"
df4.loc[df4["state"]=="오스트레일리아","state"] = "수입"
df4.loc[df4["state"]=="호주","city"] = "호주"
df4.loc[df4["state"]=="호주","state"] = "수입"
df4.loc[df4["state"]=="통가","city"] = "통가"
df4.loc[df4["state"]=="통가","state"] = "수입"
df4.loc[df4["state"]=="뉴칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴칼레","state"] = "수입"
df4.loc[df4["state"]=="뉴","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="뉴","state"] = "수입"
df4.loc[df4["state"]=="누벨칼레","city"] = "뉴칼레도니아"
df4.loc[df4["state"]=="누벨칼레","state"] = "수입"
# 기타
df4.loc[df4["state"]=="수입산","state"] = "수입"
df4.loc[df4["state"]=="기타외국","city"] = "미상"
df4.loc[df4["state"]=="기타외국","state"] = "수입"

In [120]:
df4[df4['state']=='서해안']

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [121]:
df4['state'].unique()

array(['경북', '전남', '수입', '충남', '경기', '충북', '강원', '세종', '경남', '전북', '제주',
       '인천', '대구', '서울', '광주', '대전', '부산', '국산'], dtype=object)

In [122]:
# city
# 우선 결측치 제거
df4.loc[df4["city"].isnull(),"city"] = "미상"

In [123]:
# 이후 시,군 등 제거
def name_sort(words):
  p = re.compile('[^시군]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort)

In [124]:
# \t 도 제거
def name_sort2(words):
  p = re.compile('[^\t]+')
  word = re.search(p,words)
  return word.group()

df4["city"] = df4["city"].apply(name_sort2)

In [125]:
df4['city'].unique()

array(['청도', '무안', '나주', '함평', '중국', '홍성', '예산', '화성', '옥천', '평택', '당진',
       '용인', '충주', '괴산', '구례', '여주', '청양', '음성', '의왕', '해남', '양평', '보령',
       '홍천', '안성', '아산', '인제', '춘천', '서산', '양양', '영월', '광주', '이천', '세종',
       '부여', '상주', '천안', '청주', '성주', '담양', '문경', '김천', '함양', '여수', '청원',
       '정읍', '평창', '성남', '파주', '고창', '장흥', '논산', '보성', '포천', '서천', '제주',
       '강화', '화천', '공주', '산청', '부안', '의령', '장성', '영동', '산', '의정부', '보은',
       '고흥', '가평', '철원', '양주', '예천', '미상', '진주', '창녕', '증평', '횡성', '진천',
       '고양', '원주', '남양주', '연천', '금산', '김제', '고성', '김포', '의성', '영주', '대구',
       '영천', '무주', '청송', '안동', '경산', '위', '구리', '송파구', '오산', '하남', '강동구',
       '강서구', '양천구', '하동', '강남구', '도봉구', '경주', '양구', '남구', '광산구', '영암',
       '밀양', '화순', '제천', '진도', '칠레', '미국', '필리핀', '페루', '익산', '부천', '광양',
       '곡성', '장수', '포항', '강릉', '거창', '봉화', '정선', '삼척', '완주', '고령', '진안',
       '중구', '태안', '서울', '계양구', '흥', '태백', '광명', '계룡', '중랑구', '남원', '영덕',
       '대전', '연기', '구미', '순천', '임실', '서구', '북구',

In [129]:
# 나머지 케이스 정리

# 국내
df4.loc[df4["city"]=="기타","city"] = "미상"
## 일반시
df4.loc[df4["city"]=="위","city"] = "군위"
df4.loc[df4["city"]=="고흥(나로도)","city"] = "고흥"
df4.loc[df4["city"]=="산","city"] = "군산"
df4.loc[df4["city"]=="장항(서천)","city"] = "서천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="삼천포","city"] = "사천"
df4.loc[df4["city"]=="순천(광양)","city"] = "순천"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="대천(보령)","city"] = "보령"
df4.loc[df4["city"]=="화성(평택)","city"] = "화성"
df4.loc[df4["city"]=="흥","city"] = "시흥"
df4.loc[df4["city"]=="목포(무안)","city"] = "목포"
df4.loc[df4["city"]=="목포(무안)","state"] = "전남"
df4.loc[df4["city"]=="보성(벌교)","city"] = "보성"
df4.loc[df4["city"]=="울진(후포)","city"] = "울진"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="논","city"] = "논산"
df4.loc[df4["city"]=="순","city"] = "순창"
df4.loc[df4["city"]=="익","city"] = "익산"
df4.loc[df4["city"]=="미금","city"] = "성남"
df4.loc[df4["city"]=="주문진","city"] = "강릉"
df4.loc[df4["city"]=="전남","city"] = "무안"
df4.loc[df4["city"]=="장","city"] = "장수"
df4.loc[df4["city"]=="거제(장승포)","city"] = "거제"
df4.loc[(df4["city"]=="완")&(df4["state"]=="전북"),"city"] = "완주"
df4.loc[(df4["state"]=="전북")&(df4["city"]=="남"),"city"] = "남원"
df4.loc[(df4['prd']=="호박") & (df4['state']=='전북'),"city"] = "전주"
## 광역시
df4.loc[df4["city"]=="강서구","city"] = "서울"
df4.loc[df4["city"]=="은평구","city"] = "서울"
df4.loc[df4["city"]=="강동구","city"] = "서울"
df4.loc[df4["city"]=="마포구","city"] = "서울"
df4.loc[df4["city"]=="중구","city"] = "서울"
df4.loc[df4["city"]=="송파구","city"] = "서울"
df4.loc[df4["city"]=="도봉구","city"] = "서울"
df4.loc[df4["city"]=="중랑구","city"] = "서울"
df4.loc[df4["city"]=="양천구","city"] = "서울"
df4.loc[df4["city"]=="성동구","city"] = "서울"
df4.loc[df4["city"]=="강남구","city"] = "서울"
df4.loc[df4["city"]=="동대문구","city"] = "서울"
df4.loc[df4["city"]=="영등포구","city"] = "서울"
df4.loc[df4["city"]=="영등포","city"] = "서울"
df4.loc[df4["city"]=="서초구","city"] = "서울"
df4.loc[df4["city"]=="구로구","city"] = "서울"
df4.loc[df4["city"]=="동대문","city"] = "서울"
df4.loc[df4["city"]=="금천구","city"] = "서울"
df4.loc[df4["city"]=="동작구","city"] = "서울"
df4.loc[df4["city"]=="용산구","city"] = "서울"
df4.loc[df4["city"]=="광진구","city"] = "서울"

df4.loc[df4["city"]=="광산구","city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="남구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="북구"),"city"] = "광주"
df4.loc[(df4["state"]=="광주")&(df4["city"]=="서구"),"city"] = "광주"
df4.loc[df4["city"]=="동구","city"] = "광주"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="남구"),"city"] = "인천"
df4.loc[(df4["state"]=="인천")&(df4["city"]=="서구"),"city"] = "인천"
df4.loc[df4["city"]=="미추홀","city"] = "인천"
df4.loc[df4["city"]=="부평구","city"] = "인천"
df4.loc[df4["city"]=="남동구","city"] = "인천"
df4.loc[df4["city"]=="미추홀구","city"] = "인천"
df4.loc[df4["city"]=="계양구","city"] = "인천"
df4.loc[df4["city"]=="마산(고성,진해)","city"] = "창원"
df4.loc[(df4["state"]=="부산")&(df4["city"]=="북구"),"city"] = "부산"
df4.loc[df4["city"]=="오포","city"] = "부산"
df4.loc[df4["city"]=="부산진","city"] = "부산"
df4.loc[df4["city"]=="사상구","city"] = "부산"
df4.loc[df4["city"]=="부산진구","city"] = "부산"
df4.loc[df4["city"]=="북제주","city"] = "제주"
df4.loc[df4["city"]=="남제주","city"] = "제주"

# 해외
## 아시아
df4.loc[df4["city"]=="러","city"] = "러시아"
df4.loc[df4["city"]=="말레이지아","city"] = "말레이시아"
df4.loc[df4["city"]=="타이","city"] = "태국"
df4.loc[df4["city"]=="인도네","city"] = "인도네시아"
## 아메리카
df4.loc[df4["city"]=="멕","city"] = "멕시코"
## 아프리카
df4.loc[df4["city"]=="남아공산","city"] = "남아공"
## 유럽
## 오세아니아

In [130]:
df4[df4["city"]=="완"]

,prd,scale,price,origin,eco,reg_date,new_class,price_kg,state,city


In [131]:
# 기존 city 리스트와 비교
ex_city = pd.read_csv("../cities.csv")
df4[~df4['city'].isin(ex_city['city'])]['city'].unique()

array(['우크라이나'], dtype=object)

In [132]:
# 리스트 최신화
new_city = pd.DataFrame(df4['city'].unique(),columns=['city'])
cities = ex_city.append(new_city,ignore_index=True)

/var/folders/nq/s4zx76p53d11lnr0t1dxpxh00000gn/T/ipykernel_2444/1856950528.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  cities = ex_city.append(new_city,ignore_index=True)


In [133]:
cities.drop(columns=['Unnamed: 0'],inplace=True)
cities.drop_duplicates(inplace=True)
cities.reset_index(inplace=True,drop=True)

In [134]:
cities.to_csv('../cities.csv')

In [135]:
# 처리 완료하였으니 df4는 백업으로 두고 df5 활용
df5 = df4.drop(columns='origin')

# 전처리 결과

# 파일 저장

In [137]:
df5.to_csv('../data_proceed/202110_proceed.csv')